## Finite Difference - European Options

Crank-Nicolson

In [41]:
from typing import Tuple, Dict
import numpy as np
from scipy.stats import norm

K = 100
r = 0.1
q = 0.15
sigma = 0.2
T = 1.0

S0 = 200
M  = 300 # Space grid points
N = 10_000 # Time grid points

In [ ]:
# Central difference for t 
# Central difference for S

def bs_call_pde_fd_cn(
        S0: float, 
        K: float, 
        r: float, 
        q: float, 
        sigma: float, 
        T: float,
        rannacher_steps: int , # number of initial half implicit steps 
        M: int = 300,         
        N: int = 300,
        ) -> Tuple[float, np.ndarray, np.ndarray, np.ndarray]:
    
    Smax = S0*5
    
    # Space grid -> M --> i
    S = np.linspace(start=0.0, stop=Smax, num=M+1)
    
    # Time grid -> N --> n
    t = np.linspace( start=0.0, stop=T, num=N+1)
    dt = t[1]-t[0]
    
    # Terminal condition i.e. value of option at maturity
    # This is the starting point for the backward evolution.
    V = np.maximum(S - K, 0.0)  

    # Store the full surface for visualization
    V_surface = np.zeros((M+1, N+1))
    V_surface[:, -1] = V.copy()

    # Coefficient for Tridiagonal Matrix
    i = np.arange(start=1, stop=M)
    alpha = 0.25*dt*(sigma**2*(i**2) - (r-q)*i)
    beta  = -0.5*dt*(sigma**2*(i**2) + r)
    gamma = 0.25*dt*(sigma**2*(i**2) + (r-q)*i)

    # AVn=BVn+1
    # Asub​V^n_i−1​+Adiag​V^n_i​+Asup​V^n_i+1​ = Bsub​V^n+1_i−1​+Bdiag​V^n+1_n​+Bsup​V^n+1_i+1​.
    A_sub = -alpha.copy()
    A_diag = 1 - beta.copy()
    A_sup = -gamma.copy()

    B_sub = alpha.copy()
    B_diag = 1 + beta.copy()
    B_sup = gamma.copy()

    # --- Precompute fully implicit matrices for Rannacher half-steps ---
    # i.e dt/2
    dt_half  = 0.5 * dt
    alpha_imp = -0.5*dt_half*(sigma**2*(i**2) - (r - q)*i)
    beta_imp  = 1 +  dt_half*(sigma**2*(i**2) + r)
    gamma_imp = -0.5*dt_half*(sigma**2*(i**2) + (r - q)*i)
    A_imp = np.zeros((M - 1, M - 1))
    np.fill_diagonal(A_imp, beta_imp)
    np.fill_diagonal(A_imp[1:], alpha_imp[1:])
    np.fill_diagonal(A_imp[:, 1:], gamma_imp[:-1])

    from numpy.linalg import solve
    # i --> space S (M)
    # n --> time t
    
    # We move backwards in time, starting from maturity (N) to now (0)
    for n in range(N, 0, -1):
        # At each iteration, we have the grid V^n+1 --> known from the next/later time
        # We compute V_n --> the unknown, earlier time by solving a linear system.

        # Build the right hand side of Crank-Nicholson (EXPLICIT)
        rhs = B_sub*V[i-1] + B_diag*V[i] + B_sup*V[i+1]
        
        # Boundary: V0 and Vmax are the known boundary values at this time level t_n
        # Becomes smaller and smaller each iteration.
        t_now = (n-1)*dt
        # V(t_n,0)
        V0 = 0.0
        # V(t_n,S_max) --> linear, Using last value of S
        Vmax = S[-1]*np.exp(-q*(T - t_now)) - K*np.exp(-r*(T - t_now))

        # We "correct" the system to account for known edge values. 
        # Without this, the first and last interior equations would incorrectly include the unknown boundaries.
        rhs[0]  -= A_sub[0]*V0
        rhs[-1] -= A_sup[-1]*Vmax

        # Build IMPLICIT matrix
        # M-1 because solve only for interior nodes i=1...M-1 
        # since boundaries V0 and VM are known and excluded from the system 
        A = np.zeros((M-1, M-1))
        np.fill_diagonal(A, A_diag) # Central point
        np.fill_diagonal(A[1:], A_sub[1:]) # Forward neighbor
        np.fill_diagonal(A[:,1:], A_sup[:-1]) # Backward neighbor


        # Rannacher smoothing: use implicit half-steps for first few iterations
        if n > N - rannacher_steps:

            # Implicit damping:
            # Instead of going simply to t_now (t_n)
            # there is an midpoint so
            # we from from t_n+1 to t_n+1/2
            # then from t_n+1/2 to t_n

            # times
            dt_half  = 0.5 * dt
            t_mid    = t_now + dt_half       # first half-step target time (backward)
            t_final  = t_now                 # second half-step target time

            # boundaries at the two target times
            V0_mid   = 0.0
            Vmax_mid = S[-1]*np.exp(-q*(T - t_mid))  - K*np.exp(-r*(T - t_mid))

            V0_fin   = 0.0
            Vmax_fin = S[-1]*np.exp(-q*(T - t_final)) - K*np.exp(-r*(T - t_final))

            # first half-step to t_mid
            # A_imp * V^{n+1/2} = V^{n+1}
            rhs_imp = V[i].copy()

            rhs_imp[0]  -= alpha_imp[0] * V0_mid    # subtracting so it doesnt appear as unknown
            rhs_imp[-1] -= gamma_imp[-1] * Vmax_mid # subtracting so it doesnt appear as unknown
            V_half = solve(A_imp, rhs_imp)

            # second half-step to t_final
            # A_imp * V^{n} = V^{n+1/2}
            rhs_imp2 = V_half.copy()
            
            rhs_imp2[0]  -= alpha_imp[0] * V0_fin   # subtracting so it doesnt appear as unknown
            rhs_imp2[-1] -= gamma_imp[-1] * Vmax_fin # subtracting so it doesnt appear as unknown
            V_in = solve(A_imp, rhs_imp2)
        else:
            # Normal Crank–Nicolson step
            V_in = solve(A, rhs)


        # After each time n, full profile V_n=[V^0n_​,V^1_n​,…,V^M−1_n​,V^M_n​].
        V[0] = V0
        V[1:M] = V_in
        V[M] = Vmax

        V_surface[:, n-1] = V.copy()

    # Interpolate the option price at the current stock price
    # Value of option at time zero for S
    price = np.interp(S0, S, V)

    # V is just the last profile
    # V_surface contains all profiles
    
    return price, S, t, V, V_surface

In [43]:
fd_cn_price,S_cn, t_cn, V_cn, V_surface_cn = \
    bs_call_pde_fd_cn(S0, K, r, q, sigma, T, 3, M, N)

In [44]:
def greeks_from_surface(
    S0: float,
    S: np.ndarray,
    t: np.ndarray,
    V_surface: np.ndarray,
    ) -> Dict[str, float]:

    # Delta, Gamma from spatial central differences at t=0 (with interpolation).
    # Theta from forward difference in time at t=0.
        
    dt = t[1] - t[0]

    # Value at t=0
    V_t0  = V_surface[:, 0]
    # Next Value
    V_tdt = V_surface[:, 1]

    # Delta underlying
    dS = S[1] - S[0]

    # Interpolate around S0
    V_neg = np.interp(S0 - dS, S, V_t0)
    V_0 = np.interp(S0, S, V_t0)
    V_pos = np.interp(S0 + dS, S, V_t0)

    # At t=0
    Delta = (V_pos - V_neg) / (2.0 * dS)
    Gamma = (V_pos - 2.0 * V_0 + V_neg) / (dS**2)

    # Theta at t=0
    V0_t0  = V_0
    V0_tdt = np.interp(S0, S, V_tdt)
    Theta = (V0_tdt - V0_t0) / dt

    return {"Delta": float(Delta), "Gamma": float(Gamma), "Theta": float(Theta)}


In [45]:
greeks_cn: Dict[str, float] = \
    greeks_from_surface(S0, S_cn, t_cn, V_surface_cn)  # Delta, Gamma, Theta

In [46]:

def bs_call_greeks(
    S0: float,
    K: float,
    r: float,
    q: float,
    sigma: float,
    T: float
) -> Dict[str, float]:

    # d1 and d2
    d1 = (np.log(S0 / K) + (r - q + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)

    Nd1 = norm.cdf(d1)
    Nd2 = norm.cdf(d2)
    nd1 = norm.pdf(d1)

    # Price
    price = S0 * np.exp(-q * T) * Nd1 - K * np.exp(-r * T) * Nd2

    # Greeks
    delta = np.exp(-q * T) * Nd1
    gamma = np.exp(-q * T) * nd1 / (S0 * sigma * np.sqrt(T))

    # Theta
    theta = (
        - (S0 * sigma * np.exp(-q * T) * nd1) / (2 * np.sqrt(T)) # Time Decay
        - r * K * np.exp(-r * T) * Nd2                           # Dividends foregone
        + q * S0 * np.exp(-q * T) * Nd1                          # Discounting of strike
    )

    return {"Price": price, "Delta": delta, "Gamma": gamma, "Theta": theta}

In [47]:
greeks_bs = bs_call_greeks(S0, K, r, q, sigma, T)

In [48]:
print(f"{'Scheme':<15s} {'Price':>10s} {'Delta':>12s} {'Gamma':>12s} {'Theta':>12s}")
print("-" * 63)

print(f"{'FD CN - RS':<15s}"
      f"{fd_cn_price:10.6f}"
      f"{greeks_cn['Delta']:12.6f}"
      f"{greeks_cn['Gamma']:12.6f}"
      f"{greeks_cn['Theta']:12.6f}")

print(f"{'Black Scholes':<15s}"
      f"{greeks_bs['Price']:10.6f}"
      f"{greeks_bs['Delta']:12.6f}"
      f"{greeks_bs['Gamma']:12.6f}"
      f"{greeks_bs['Theta']:12.6f}")

print("-" * 63)

Scheme               Price        Delta        Gamma        Theta
---------------------------------------------------------------
FD CN - RS      81.662552    0.860286    0.000037   16.739909
Black Scholes   81.662196    0.860315    0.000035   16.741217
---------------------------------------------------------------
